In [380]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re

## nltk imports
import nltk
nltk.download('punkt')
import nltk
## nltk imports
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
!pip install spacy
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
! python -m spacy download en_core_web_sm

import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/prof.yeeyeeaung/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prof.yeeyeeaung/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prof.yeeyeeaung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

     |████████████████████████████████| 13.9 MB 8.2 MB/s eta 0:00:01    |███████████████████▋            | 8.5 MB 3.7 MB/s eta 0:00:02


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Text analysis of DOJ press releases (52 points)

For background, here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 

Here's the code the dataset owner used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

In [381]:
## first, unzip the file pset4_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()


id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [382]:
## Subset to one press release and take the string
doj["contents"] = doj["contents"].astype("string")
pharma = doj.contents[doj.id == "17-1204"].iloc[0]
##pharma


### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for .isalpha(): https://www.w3schools.com/python/ref_string_isalpha.asp
- `processtext` function here has an example of tokenizing and filtering to words where .isalpha() is true: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/06_textasdata_partII_topicmodeling.ipynb
- Part of speech tagging section of this code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partI_textmining_examplecode.ipynb



In [383]:
##A
##Restrict to alpha
import nltk
tokens = [word for word in wordpunct_tokenize(pharma) if 
          word.isalpha()]


In [384]:
##B
##Use the part of speech tagger within nltk to tag all the words  with their part of speech
tokens_pos = pos_tag(tokens) # generate part of speech tags for those tokens
tokens_pos


former        8
opioid        5
nationwide    4
addictive     3
other         3
dtype: int64

In [ ]:
##C
##Extract the adjectives and sort those adjectives from most occurrences to fewest occurrences
all_adj = [one_tok[0] for one_tok in tokens_pos
           if (one_tok[1] == "JJ") | (one_tok[1] == "JJJ")| (one_tok[1] == "JJS")]

all_adj_sort = pd.Series(all_adj).value_counts()
all_adj_sort.head(5)


## 1.2 named entity recognition (4 points)

**Resources**:

- For parts A and B: named entity recognition part of this code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partI_textmining_examplecode.ipynb
- For part E, re.search and re.findall examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/04_basicregex_solutions.ipynb

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [385]:
##A
##Extract name entities 
spacy_pharma = nlp(pharma)


['RICO', 'the Controlled Substances Act']

In [ ]:
##B
##Print unique named entities with LAW tag 
law_tag = list(set([e.text for e in spacy_pharma.ents if e.label_ == "LAW"]))
law_tag


In [ ]:
##C
#RICO means Racketeer Influenced and Corrupt Organizations Act and it might appy to a pharmaceutical kickbacks case because
#organizations involved deal use bribery, illegal cash payments, and improper practices that inflate drug practices. 


D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [386]:
##D
##Extract entities with label DATE and contain year/s
new_entities = list([e.text for e in spacy_pharma.ents if e.label_ == "DATE"])

## Regax pattern
regax = r'(?=.*(year|years))'

## Print entities
years = [i for i in new_entities if re.search(regax, i)]
years


['last year', '20 years', 'three years', 'five years', 'three years']

E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). *Hint*: you may want to use re.search or re.findall 


In [1]:
##E
##Extract parts of press release with mention of year lengths
pattern = r'^[A-Za-z,;"]\s*+(last year|20 years|three years|five years)\s*\.$'
##re.findall(pattern, pharma)

## The CEO may be facing no greater than 20 years in prison and 3 years of probation. 


## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partI_textmining_examplecode.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [388]:
##A
##Subsetting
doj_subset= doj[(doj["topics_clean"] == "Civil Rights") | (doj["topics_clean"] == "Hate Crimes") | (doj["topics_clean"] == "Project Safe Childhood")]
doj_subset.shape


(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (hint: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- You may want to use `re.escape` at some point to avoid errors relating to escape characters like ( in the press release
- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [584]:
##B
##Define a function 
def function(string):
    ## Remove name entities 
    spacy = nlp(string)
    spacy_removed = " ".join([ent.text for ent in spacy if not ent.ent_type_])

    ## Create scorer
    sent_obj = SentimentIntensityAnalyzer()
    sentiment_example = sent_obj.polarity_scores(spacy_removed)
    
    return(sentiment_example)
    
##Execute function
for column in doj_subset["contents"]:
    function(column)


{'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992}

{'neg': 0.11, 'neu': 0.825, 'pos': 0.065, 'compound': -0.891}

{'neg': 0.091, 'neu': 0.838, 'pos': 0.07, 'compound': -0.8316}

{'neg': 0.114, 'neu': 0.81, 'pos': 0.076, 'compound': -0.9037}

{'neg': 0.166, 'neu': 0.783, 'pos': 0.051, 'compound': -0.9839}

...

{'neg': 0.159, 'neu': 0.784, 'pos': 0.057, 'compound': -0.9787}

{'neg': 0.075, 'neu': 0.829, 'pos': 0.096, 'compound': 0.7003}

{'neg': 0.148, 'neu': 0.76, 'pos': 0.092, 'compound': -0.9538}

{'neg': 0.132, 'neu': 0.784, 'pos': 0.085, 'compound': -0.9866}

{'neg': 0.145, 'neu': 0.824, 'pos': 0.031, 'compound': -0.9938}

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

In [390]:
##C
##Create new dataframe to store 4 sentiment scores
df = pd.DataFrame(columns = ["neg","neu","pos", "compound"])


## Apply function to press releases and add scores to empty dataframe
for column in doj_subset["contents"]:
    scores = function(column)
    df = df.append(pd.Series(scores), ignore_index=True)

## Get filled dataframe
doj_subset_wscore = df.copy()


In [391]:
## Concatenate sentiment scores dataframe to main dataframe 
doj_subset_reset = doj_subset.reset_index()
doj_subset_wscore = pd.concat([doj_subset_reset,doj_subset_wscore], axis=1)


In [392]:
## Sort by neg
doj_subset_wscore_sort = doj_subset_wscore.sort_values(by=["neg"], ascending=False)
doj_subset_wscore_sort[["id", "contents", "neg"]].head(2)


,id,contents,neg
13,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.316
34,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.299


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [ ]:
##D
##Find mean compound sentiment score for each of the three topics
doj_subset_mean = doj_subset_wscore.groupby("topics_clean").agg({"compound" : "mean"}).reset_index()
doj_subset_mean


In [393]:
##E
#We might see variation in scores because the three different topics may have different results 
#of sentiment analysis scores and it is possible that hate crimes have the most negative compound score because
#more negative words appear under this topic.

,topics_clean,compound
0,Civil Rights,-0.127280
1,Hate Crimes,-0.932998
2,Project Safe Childhood,-0.657389


# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's more condensed code with topic modeling steps: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partII_topicmodeling_solution.ipynb
- Here's longer code with more broken-out topic modeling steps: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/06_textasdata_partII_topicmodeling.ipynb 

In [394]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [395]:
##A
##Define a function
list_stopwords = stopwords.words("english")
list_stopwords_new = list_stopwords + custom_doj_stopwords

## Initiliaze stemmer
porter = PorterStemmer()

def processfunc(string):
    ## Convert to lowercase
    corpus_lower = string.lower()
    
    try:
        nostop_listing = [word for word in wordpunct_tokenize(corpus_lower)
                          if word not in list_stopwords_new]
        
        clean_listing = [porter.stem(word) for word in nostop_listing
                         if word.isalpha() 
                         and len(word) >= 4]
        
        clean_listing_str = " ".join(clean_listing)
        
        return(clean_listing_str)
    
    except:
        return("")


In [396]:
##B
##Execute function with list comprehension
cleaned_listings = [processfunc(one_listing) for one_listing in
                   doj_subset_wscore["contents"]]


In [397]:
##C
##Store values as new column in dataframe
doj_subset_wscore["processed_text"] = cleaned_listings

## Print id, contents, and processed text for two specific cases
doj_subset_ids = doj_subset_wscore[(doj_subset_wscore["id"] == "16-718") | (doj_subset_wscore["id"] == "16-217")]
doj_subset_ids[["id", "contents", "processed_text"]]


,id,contents,processed_text
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi gener vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quickli investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi gener gupta agreement help strengthen relationship commun serv improv account offic fire weapon unlaw provid commun particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make commun safe protect sacr constitut citizen 

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

**Resources**:

- Here contains an example of applying the create_dtm function: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partII_topicmodeling_solution.ipynb


In [398]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names())
    metadata.columns = ["metadata_" + col for col in metadata.columns]
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)


In [429]:
##A
## Create a document-term matrix using the preprocessed press releases
doj_subset_wscore["contents_lower"] = doj_subset_wscore["processed_text"].str.lower()

dtm_pre = create_dtm(list_of_strings= doj_subset_wscore.contents_lower,
                     metadata = doj_subset_wscore[["id", "compound","topics_clean"]])

dtm_pre


,index,metadata_id,metadata_compound,metadata_topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealou,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,0,17-1235,-0.9920,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,15-1522,-0.8910,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,16-213,-0.8316,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,16-381,-0.9037,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,14-464,-0.9839,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,712,09-368,-0.9787,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,713,18-775,0.7003,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,714,12-596,-0.9538,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,715,18-359,-0.9866,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [433]:
## Function get top words 
def get_topwords(subset):

    dtm_pre_termsonly = subset[[col for col in subset.columns if
                                "metadata" not in col and col != "index"]].sum(axis=0)
    
    dtm_sorted = dtm_pre_termsonly.sort_values(ascending = False)
    
    return dtm_sorted.head(10)


In [437]:
##B
## Dataset with highest 5% of compound scores  (most positive)
value_h5 = dtm_pre["metadata_compound"].quantile(0.95)
doj_subset_hcompound = dtm_pre[dtm_pre["metadata_compound"] >= value_h5]

# Get top words
print("Top 5% compound scores")
get_topwords(doj_subset_hcompound)


Top 5% compound scores


agreement     188
enforc        121
commun        113
disabl        113
ensur         109
state         106
settlement     92
student        91
polic          88
servic         83
dtype: int64

In [438]:
##C
## Dataset with lowest 5% of compound scores  (most negative)
value_l5 = dtm_pre["metadata_compound"].quantile(0.05)
doj_subset_lcompound = dtm_pre[dtm_pre["metadata_compound"] <= value_l5]

#Get top words
print("Lowest 5% compound scores")
get_topwords(doj_subset_lcompound)


Lowest 5% compound scores


assault     201
crime       176
victim      176
defend      140
hate        139
offic       120
conspir     115
american    114
sentenc     111
charg       108
dtype: int64

In [440]:
##D
## Dataset subset to Civil Rights
doj_subset_rights = dtm_pre[dtm_pre["metadata_topics_clean"] == "Civil Rights"]
print("Top 10 words: Civil Rights")
get_topwords(doj_subset_rights)

## Dataset subset to Project Safe Childhood
doj_subset_child = dtm_pre[dtm_pre["metadata_topics_clean"] == "Project Safe Childhood"]
print("Top 10 words: Project Safe Childhood")
get_topwords(doj_subset_child)

## Dataset subset to Hate Crimes
doj_subset_crimes = dtm_pre[dtm_pre["metadata_topics_clean"] == "Hate Crimes"]
print("Top 10 words: Hate Crimes")
get_topwords(doj_subset_crimes)


Top 10 words: Civil Rights


offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64

Top 10 words: Project Safe Childhood


child          1022
exploit         701
sexual          572
safe            479
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
dtype: int64

Top 10 words: Hate Crimes


victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [455]:
##A
##Estimate topic model
tokenized_text = [wordpunct_tokenize(one_text) for one_text in cleaned_listings]

#Create dictionary 
text_proc_dict = corpora.Dictionary(tokenized_text)

#Create corpus from dictionary
corpus_fromdict_proc = [text_proc_dict.doc2bow(one_text)
                        for one_text in tokenized_text]

#Estimate model
ldamod_proc = gensim.models.ldamodel.LdaModel(corpus_fromdict_proc,
                                              num_topics = 3, id2word=text_proc_dict,
                                              passes=6, alpha = 'auto',
                                              per_word_topics = True, random_state = 91988)


In [456]:
##B
##Print top 15 words 
topics = ldamod_proc.print_topics(num_words = 15)
for topic in topics:
    print(topic)
    

(0, '0.022*"child" + 0.015*"exploit" + 0.013*"sexual" + 0.010*"safe" + 0.010*"childhood" + 0.010*"project" + 0.010*"prosecut" + 0.010*"pornographi" + 0.010*"crimin" + 0.009*"children" + 0.009*"sentenc" + 0.008*"victim" + 0.008*"year" + 0.008*"feder" + 0.007*"investig"')
(1, '0.008*"feder" + 0.007*"said" + 0.007*"victim" + 0.006*"crime" + 0.006*"american" + 0.006*"prosecut" + 0.006*"offic" + 0.006*"sentenc" + 0.006*"charg" + 0.006*"hate" + 0.006*"guilti" + 0.005*"today" + 0.005*"violat" + 0.005*"enforc" + 0.005*"court"')
(2, '0.012*"hous" + 0.010*"disabl" + 0.008*"said" + 0.007*"discrimin" + 0.007*"defend" + 0.006*"alleg" + 0.006*"victim" + 0.006*"today" + 0.006*"state" + 0.006*"feder" + 0.005*"violat" + 0.005*"enforc" + 0.005*"gener" + 0.005*"fair" + 0.005*"agreement"')


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code (`Additional summaries of topics and documents`) contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/06_textasdata_partII_topicmodeling_solution.ipynb
- If you're getting errors, use shape, len, and other commands to check the dimensionality of things at different steps 

In [463]:
##A
##Get doc-level topic probabilities
topic_probs_bydoc =[ldamod_proc.get_document_topics(item, minimum_probability = 0) for item in corpus_fromdict_proc]
print(len(topic_probs_bydoc))

##Assert statment to check
assert len(topic_probs_bydoc) == len(doj_subset_wscore), "Number of rows are different"


717


In [473]:
##B
##Add topic probabilities to dataframe as columns 
one_list_tup = topic_probs_bydoc[0]

## Create a long type dataframe 
topic_probs_bydoc_long = pd.DataFrame([t for lst in topic_probs_bydoc for t in lst],
                                      columns = ['topic', 'probability'])

## add id var- for the number of topics
topic_probs_bydoc_long['doc_id'] = list(np.concatenate([[one_id] * 
                                                        3 for one_id in doj_subset_wscore.id]).flat)

## Pivot to wide format
topic_probs_bydoc_wide = pd.pivot_table(topic_probs_bydoc_long, index = ['doc_id'],
                        columns = ['topic']).reset_index().reset_index(drop = True)

topic_probs_bydoc_wide.columns = ['doc_id'] + ["topic_" + str(i) for i in np.arange(0, 3)]


## Merge with original data 
topic_wmeta = pd.merge(topic_probs_bydoc_wide,
                       doj_subset_wscore,
                       left_on = 'doc_id',
                       right_on = 'id')
 

##Create a column, top_topic, that reflects each document to its highest-probability topic 
topic_wmeta['toptopic'] = topic_wmeta[[col for col in topic_wmeta.columns if 
                                       "topic_" in col]].idxmax(axis=1)

topic_wmeta.sample(n = 5, random_state = 555)


doc_id   topic_0   topic_1   topic_2  index       id  \
259  15-1263  0.540345  0.459379  0.000276   4215  15-1263   
429   16-252  0.000466  0.999019  0.000515   3581   16-252   
374  16-1229  0.000205  0.745421  0.254374   7125  16-1229   
82   11-1328  0.000662  0.998606  0.000732   8491  11-1328   
239   15-015  0.000099  0.999792  0.000109  11763   15-015   

                                                                                                                                                                                title  \
259                                              Former Mamou, Louisiana, Police Chief Sentenced, Second Former Police Chief Pleads Guilty to Firing Taser at Non-Combative Prisoners   
429                                                                                          Former Alcorn State University Police Officer Pleads Guilty to Assaulting Former Student   
374  Justice Department Seeks to Intervene in Lawsuit Alleging Race Discrimination and Retaliation by Pocomoke City, Maryland, the Worcester County Sheriff and the State of Maryland   
82                                                                             Member of United Aryan Brotherhood Pleads Guilty to a Hate Motivated Assault of Jewish Inmate in Texas   
239                                                           Two Brandon, Mississippi, Men Plead Guilty for Committing Hate Crimes Against African Americans in Jackson, Mississippi   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [486]:
##C
##Summarize the topic proportions for each of the topics_clean 
pd.crosstab(topic_wmeta["topics_clean"], topic_wmeta["toptopic"], normalize = "index")


toptopic,topic_0,topic_1,topic_2
topics_clean,,,
Civil Rights,0.059801,0.438538,0.501661
Hate Crimes,0.000000,0.735772,0.264228
Project Safe Childhood,0.993750,0.006250,0.000000


In [ ]:
##D
#Some might map on more cleanly to a topic than others based on the words and phrases that are highlighted and 
#the corresponding topic proportions. For example, id 15-1263 is mapped to topic 0, Civil Rights, although the 
#topic proportion for Hate Crimes is a close second because the action committed was not very violent or extreme; 
#on the other hand, case 16-252 is clearly mapped to topic 1, Hate Crime, because words such as "elbowed" and "threw"
#indicate the more violent and physical nature of the action. 


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [578]:
##A 
## Function to return string of bigrams
def create_bigram_onedoc(string):
    Bigrams = []
    split = string.split()
    bg = list(nltk.bigrams(split))
    for bigram in bg:
        Bigrams.append(("_".join([w +"" for w in bigram])).strip())
    str1 = " ".join(Bigrams)
    return str1
    
## Create new column to store bigrams
processed_text_bigrams = [create_bigram_onedoc(one_listing) for one_listing in
                          doj_subset_wscore["processed_text"]]


doj_subset_wscore["processed_text_bigrams"] = processed_text_bigrams


In [579]:
##B
##Print the id, processed_text, and processed_text_bigram columns for press release with id = 16-217
example = doj_subset_wscore[doj_subset_wscore["id"] == "16-217"]
example[["id", "processed_text", "processed_text_bigrams"]]


,id,processed_text,processed_text_bigrams
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi gener vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quickli investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi gener gupta agreement help strengthen relationship commun serv improv account offic fire weapon unlaw provid commun particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make commun safe protect sacr constitut citizen said ferrer oversight commun agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss vote settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_offic offic_involv involv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_gener gener_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_offic offic_involv involv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_offic offic_involv involv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_offic offic_involv involv_shoot shoot_effect effect_quickli quickli_investig investig_offic offic_involv involv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_gener gener_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_commun commun_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_commun commun_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_commun commun_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_commun commun_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_made made_ap

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [580]:
##C
##Create document term matrix 
dtm_bigram = create_dtm(list_of_strings= doj_subset_wscore.processed_text_bigrams,
                        metadata = doj_subset_wscore[["id", "compound","topics_clean"]])

dtm_bigram

,index,metadata_id,metadata_compound,metadata_topics_clean,aaron_ford,aaron_latham,aaron_mcgrath,aaron_parrish,aaron_polster,aaron_rice,...,zone_religi,zone_request,zone_restrict,zone_student,zone_time,zone_varianc,zunggeemog_noel,zunggeemog_prompt,zunggeemog_write,zwengel_princeton
0,0,17-1235,-0.9920,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,15-1522,-0.8910,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,16-213,-0.8316,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,16-381,-0.9037,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,14-464,-0.9839,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,712,09-368,-0.9787,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,713,18-775,0.7003,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,714,12-596,-0.9538,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,715,18-359,-0.9866,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [581]:
##D
##Print dimensions of dtm matrix 
print("Dimensions of unigram matrix")
dtm_pre.shape

print("Dimensions of bigram matrix")
dtm_bigram.shape

#The bigram matrix has more dimensions than the unigram matrix because bigram matrix takes 
##more unique sets of adjacent words.


Dimensions of unigram matrix


(717, 6932)

Dimensions of bigram matrix


(717, 72763)

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [583]:
##E
## Dataset subset to Civil Rights
doj_subset_rights2 = dtm_bigram[dtm_bigram["metadata_topics_clean"] == "Civil Rights"]
print("Top 10 words: Civil Rights")
get_topwords(doj_subset_rights2)

## Dataset subset to Project Safe Childhood
doj_subset_child2 = dtm_bigram[dtm_bigram["metadata_topics_clean"] == "Project Safe Childhood"]
print("Top 10 words: Project Safe Childhood")
get_topwords(doj_subset_child2)

## Dataset subset to Hate Crimes
doj_subset_crimes2 = dtm_bigram[dtm_bigram["metadata_topics_clean"] == "Hate Crimes"]
print("Top 10 words: Hate Crimes")
get_topwords(doj_subset_crimes2)


Top 10 words: Civil Rights


fair_hous         231
deputi_gener      221
princip_deputi    221
vanita_gupta      202
gupta_head        200
gener_vanita      199
said_princip      186
unit_state        156
nation_origin     143
consent_decre     128
dtype: int64

Top 10 words: Project Safe Childhood


safe_childhood       474
project_safe         472
child_pornographi    450
child_exploit        281
sexual_exploit       223
exploit_children     200
plead_guilti         197
exploit_obscen       176
obscen_section       175
child_sexual         174
dtype: int64

Top 10 words: Hate Crimes


hate_crime          379
african_american    367
plead_guilti        275
year_prison         161
special_agent       124
racial_motiv        114
thoma_perez         111
grand_juri          101
perez_gener          95
said_thoma           91
dtype: int64

# 4. Optional extra credit 1 (3 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [23]:
# your code here 

# 5. Optional extra credit 2 (6 points)

Review the scraping code here: https://github.com/jbencina/dojreleases/blob/master/scraper.py
    
Write code to scrape press releases from more recent years from the DOJ website than the years available in the combined.json (which stops around 2018 or 2019) and produce a visualization of how top words or topics in the press releases changed across the Trump administration (2016-December 2020) versus Biden administration (January 2021 onwards)  

In [24]:
# your code here